In [1]:
from src.models import Simulation, Strategy, Game
import numpy as np
import pandas as pd

In [2]:
class BenoulliProcess(Game):
    def __init__(self, title: str = "Bernoulli Process", p: float = 0.5):
        super().__init__(title)
        self.p = p

    def play_round(self, game_history):
        return 1 if np.random.rand() < self.p else -1
    
class FixedBettingStrategy(Strategy):
    def __init__(self,bet_amount: float, name: str = "Fixed Bet Strategy"):
        super().__init__(name)
        self.bet_amount = bet_amount

    def decide_bet(self, game_history):
        return self.bet_amount
    
class DoubleOnLossStrategy(Strategy):
    def __init__(self, base_bet: float, name: str = "Double On Loss Strategy"):
        super().__init__(name)
        self.base_bet = base_bet
        self.current_bet = base_bet

    def decide_bet(self, game_history):
        last = game_history.last_record()
        print("Last game record:", last)
        if not game_history.is_empty() and last["outcome"] == -1:
            self.current_bet *= 2
        else:
            self.current_bet = self.base_bet
        return self.current_bet
    
sim = Simulation(
    game=BenoulliProcess(p=0.5),
    strategies=[
        FixedBettingStrategy(bet_amount=10),
        DoubleOnLossStrategy(base_bet=5)
    ],
    max_bet=1000000,
    start_value=1000
)

results = sim.run(rounds=1000)

/home/desire/Desktop/Martingale/src/models.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.records = pd.concat([self.records, new_record], ignore_index=True)


Last game record: Series([], dtype: object)
Last game record: outcome        -1.0
bet             5.0
total_gain    995.0
Name: 0, dtype: float64
Last game record: outcome          1.0
bet             10.0
total_gain    1005.0
Name: 1, dtype: float64
Last game record: outcome          1.0
bet              5.0
total_gain    1010.0
Name: 2, dtype: float64
Last game record: outcome         -1.0
bet              5.0
total_gain    1005.0
Name: 3, dtype: float64
Last game record: outcome          1.0
bet             10.0
total_gain    1015.0
Name: 4, dtype: float64
Last game record: outcome         -1.0
bet              5.0
total_gain    1010.0
Name: 5, dtype: float64
Last game record: outcome          1.0
bet             10.0
total_gain    1020.0
Name: 6, dtype: float64
Last game record: outcome         -1.0
bet              5.0
total_gain    1015.0
Name: 7, dtype: float64
Last game record: outcome          1.0
bet             10.0
total_gain    1025.0
Name: 8, dtype: float64
Last game reco

/home/desire/Desktop/Martingale/src/models.py:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.records = pd.concat([self.records, new_record], ignore_index=True)


In [6]:
results["Fixed Bet Strategy"].compute_statistics()

{'total_rounds': 1000,
 'final_gain': np.float64(1080.0),
 'average_bet': np.float64(10.0),
 'max_bet': np.float64(10.0),
 'min_bet': np.float64(10.0),
 'max_drawdown': np.float64(420.0)}

In [9]:
results["Fixed Bet Strategy"].history

,outcome,bet,total_gain
0,1.0,10.0,1010.0
1,-1.0,10.0,1000.0
2,1.0,10.0,1010.0
3,1.0,10.0,1020.0
4,-1.0,10.0,1010.0
...,...,...,...
995,-1.0,10.0,1080.0
996,-1.0,10.0,1070.0
997,1.0,10.0,1080.0
998,1.0,10.0,1090.0


In [7]:
results["Double On Loss Strategy"].compute_statistics()

{'total_rounds': 1000,
 'final_gain': np.float64(3510.0),
 'average_bet': np.float64(15.88),
 'max_bet': np.float64(320.0),
 'min_bet': np.float64(5.0),
 'max_drawdown': np.float64(315.0)}

In [10]:
results["Double On Loss Strategy"].history

,outcome,bet,total_gain
0,-1.0,5.0,995.0
1,1.0,10.0,1005.0
2,1.0,5.0,1010.0
3,-1.0,5.0,1005.0
4,1.0,10.0,1015.0
...,...,...,...
995,-1.0,5.0,3515.0
996,-1.0,10.0,3505.0
997,1.0,20.0,3525.0
998,-1.0,5.0,3520.0
